In [ ]:
import tensorflow as tf
# Load the Keras model
model = tf.keras.models.load_model("my_model.h5")

# Convert to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the converted model
with open("model.tflite", "wb") as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: C:\Users\luzif\AppData\Local\Temp\tmpem1lxt9e\assets


INFO:tensorflow:Assets written to: C:\Users\luzif\AppData\Local\Temp\tmpem1lxt9e\assets


Saved artifact at 'C:\Users\luzif\AppData\Local\Temp\tmpem1lxt9e'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 7), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 22), dtype=tf.float32, name=None)
Captures:
  2195595485136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2196302987088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2196302986320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2196302983632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2196302985936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2196302986896: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [ ]:
##testing thhe created tflite file to check if it still works

import tensorflow as tf
import numpy as np

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="model.tflite")

# Allocate tensors
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Prepare input data (ensure the shape matches)
input_shape = input_details[0]['shape']  # Example: [1, 7]
print(f"Expected input shape: {input_shape}")

# Ensure input_data matches the expected shape and type
input_data = np.array([[30, 25, 31, 26.31270635, 98.62048026, 5.804965067, 208.1181381]], dtype=np.float32)

# Verify the input shape
if input_data.shape != tuple(input_shape):
    print(f"Error: Expected input shape {input_shape}, but got {input_data.shape}")
else:
    # Set the input tensor
    interpreter.set_tensor(input_details[0]['index'], input_data)

    # Run the inference
    interpreter.invoke()

    # Get the output tensor
    output_data = interpreter.get_tensor(output_details[0]['index'])
    print("Output:", output_data)


Expected input shape: [1 7]
Output: [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]


In [ ]:
# Convert .tflite model to a C++ header file

def convert_tflite_to_cpp_header(model_filename, header_filename):
    with open(model_filename, "rb") as f:
        model_data = f.read()

    with open(header_filename, "w") as header_file:
        header_file.write("#ifndef MODEL_H\n")
        header_file.write("#define MODEL_H\n\n")
        header_file.write("const uint8_t g_model[] = {\n")

        # Write byte values as comma-separated values in the C++ header
        for i in range(0, len(model_data), 12): 
            header_file.write("  ")
            header_file.write(", ".join(f"0x{byte:02x}" for byte in model_data[i:i+12]))
            header_file.write(",\n")

        header_file.write("};\n\n")
        header_file.write("#endif  // MODEL_H\n")

convert_tflite_to_cpp_header('model.tflite', 'crop_rec_model.h')
